In [1]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.parse.corenlp import CoreNLPDependencyParser
import xml.etree.ElementTree as ET
import nltk
from nltk.corpus import stopwords
import string
sw = stopwords.words('english')

In [2]:
path = '../../data/Train'

In [3]:
def sum_one(dictionary, key):
    if key in dictionary:
        dictionary[key] += 1
    else:
        dictionary[key] = 1

In [4]:
my_parser = CoreNLPDependencyParser(url="http://localhost:9000")

drug_dict = {'effect': 
                    {'clue_words':{},
                     'sentence_words':{},
                     'e1_under':{},
                     'e2_under':{},
                     'e1_under_e2':0,
                     'e1_e2_under_same_verb':0,
                     'e1_e2_under_same_word':0,
                     'e2_under_e1':0,
                     'rel': {},
                     'total':0
                    },
                 'mechanism':
                    {'clue_words':{},
                     'sentence_words':{},
                     'e1_under':{},
                     'e2_under':{},
                     'e1_under_e2':0,
                     'e1_e2_under_same_verb':0,
                     'e1_e2_under_same_word':0,
                     'e2_under_e1':0,
                     'rel': {},
                     'total':0
                    },
                 'int':
                    {'clue_words':{},
                     'sentence_words':{},
                     'e1_under':{},
                     'e2_under':{},
                     'e1_under_e2':0,
                     'e1_e2_under_same_verb':0,
                     'e1_e2_under_same_word':0,
                     'e2_under_e1':0,
                     'rel': {},
                     'total':0
                    },
                 'advise':
                    {'clue_words':{},
                     'sentence_words':{},
                     'e1_under':{},
                     'e2_under':{},
                     'e1_under_e2':0,
                     'e1_e2_under_same_verb':0,
                     'e1_e2_under_same_word':0,
                     'e2_under_e1':0,
                     'rel': {},
                     'total':0
                    },
                 'none':
                    {'clue_words':{},
                     'sentence_words':{},
                     'e1_under':{},
                     'e2_under':{},
                     'e1_under_e2':0,
                     'e1_e2_under_same_verb':0,
                     'e1_e2_under_same_word':0,
                     'e2_under_e1':0,
                     'rel': {},
                     'total':0
                    }
                }
total_pairs = 0
total_interactions = 0

for filename in os.listdir(path):
    print(filename)
    fullname = os.path.join(path, filename)
    tree = ET.parse(fullname)
    root = tree.getroot() 
        
    for s in root.findall('sentence'):
        sid = s.get('id') # get sentence id
        stext = s.get('text') # get sentence text
        
        if stext == "":
            continue
        
        
        # load sentence entities into a dictionary
        entities = {}
        ents = s.findall('entity')
        for e in ents :
            id = e.get('id')
            offs = e.get('charOffset').split("-")
            entities[id] = offs
        
        mytree, = my_parser.raw_parse(stext)
    
        pairs = s.findall('pair')
        for p in pairs:
            id_e1 = p.get('e1')
            id_e2 = p.get('e2')
            ddi = p.get('ddi')
        
            total_pairs += 1
            start1 = int(entities[id_e1][0].split(';')[0])
            end1 = int(entities[id_e1][1].split(';')[0])
            
            start2 = int(entities[id_e2][0].split(';')[0])
            end2 = int(entities[id_e2][1].split(';')[0])
            
            entity1 = stext[start1:end1+1]
            entity2 = stext[start2:end2+1]
        
            
            if ddi == "true":
                total_interactions += 1
                ddi_type = p.get('type')
                if ddi_type is None:
                    ddi_type = 'none'
            else:
                ddi_type = 'none'
                
            # Total
            drug_dict[ddi_type]['total'] += 1
            
            sentence_between = [word for word in word_tokenize(stext[end1+1:start2]) if word not in sw and word not in string.punctuation]
            
            for word in sentence_between:         
                sum_one(drug_dict[ddi_type]['clue_words'], word)

            all_other_sentence = [word for word in word_tokenize(stext[:start1] + ' ' + stext[end1+1:start2] + ' ' + stext[end2+1:]) if word not in sw and word not in string.punctuation]
            for word in all_other_sentence:
                sum_one(drug_dict[ddi_type]['sentence_words'], word)
                
                
            head1 = ''
            head2 = ''
            tag1 = ''
            tag2 = ''
            for k in mytree.nodes:
                if mytree.nodes[k]['word'] == entity1 or mytree.nodes[k]['lemma'] == entity1:
                    h1 = mytree.nodes[k]['head']
                    head1 = mytree.nodes[h1]['lemma']
                    tag1 = mytree.nodes[h1]['tag'][0].lower()
                    
                elif mytree.nodes[k]['word'] == entity2 or mytree.nodes[k]['lemma'] == entity2:
                    h2 = mytree.nodes[k]['head']
                    head2 = mytree.nodes[h2]['lemma']
                    tag2 = mytree.nodes[h2]['tag'][0].lower()
            
            if head1 == head2 and tag1 == 'v' and tag2 == 'v':
                drug_dict[ddi_type]['e1_e2_under_same_verb'] += 1
            
            if head1 == head2:
                drug_dict[ddi_type]['e1_e2_under_same_word'] += 1
                
            if head1 == entity2:
                drug_dict[ddi_type]['e1_under_e2'] += 1
            
            if head2 == entity1:
                drug_dict[ddi_type]['e2_under_e1'] += 1
            
            
            sum_one(drug_dict[ddi_type]['e1_under'], head1) 
            sum_one(drug_dict[ddi_type]['e2_under'], head2)

10226677.xml
10319155.xml
1089034.xml
1089817.xml
10959916.xml
10978746.xml
10978751.xml
11064383.xml
11085305.xml
11085328.xml
11085333.xml
11085336.xml
1109248.xml
11115418.xml
11120981.xml
11121387.xml
11121884.xml
11125022.xml
11125235.xml
1113260.xml
11137320.xml
11137351.xml
11137703.xml
11144988.xml
11148572.xml
11151029.xml
11152438.xml
11152855.xml
1115340.xml
1115367.xml
1115445.xml
11154900.xml
11155575.xml
1115721.xml
11158747.xml
11160777.xml
1116650.xml
11166779.xml
11170315.xml
11172561.xml
11178969.xml
11180036.xml
11180037.xml
11180038.xml
11180040.xml
11181395.xml
11197581.xml
11197767.xml
11197768.xml
11198959.xml
11199955.xml
11206047.xml
11206048.xml
11206187.xml
11206417.xml
11210403.xml
11210404.xml
11210678.xml
11211460.xml
11213358.xml
11213561.xml
11213850.xml
11217867.xml
11217868.xml
11218973.xml
11219023.xml
11271411.xml
1167743.xml
1167797.xml
15825309.xml
15825391.xml
15825399.xml
15830476.xml
15986716.xml
163470.xml
16649322.xml
16649344.xml
19489169.xml

Fentanyl_ddi.xml
Fexofenadine_ddi.xml
Finasteride_ddi.xml
Flecainide_ddi.xml
Floxuridine_ddi.xml
Fludarabine_ddi.xml
Fludrocortisone_ddi.xml
Flumazenil_ddi.xml
Fluoxetine_ddi.xml
Fluoxymesterone_ddi.xml
Flupenthixol_ddi.xml
Flurbiprofen_ddi.xml
Flutamide_ddi.xml
Fluvoxamine_ddi.xml
Folic_Acid_ddi.xml
Follitropin_beta_ddi.xml
Fomepizole_ddi.xml
Fondaparinux_sodium_ddi.xml
Formoterol_ddi.xml
Foscarnet_ddi.xml
Fosfomycin_ddi.xml
Fosphenytoin_ddi.xml
Frovatriptan_ddi.xml
Fulvestrant_ddi.xml
Furosemide_ddi.xml
Gabapentin_ddi.xml
Gadodiamide_ddi.xml
Galsulfase_ddi.xml
Ganciclovir_ddi.xml
Gemfibrozil_ddi.xml
Gemifloxacin_ddi.xml
Gemtuzumab_ozogamicin_ddi.xml
Glatiramer_Acetate_ddi.xml
Glibenclamide_ddi.xml
Glimepiride_ddi.xml
Glipizide_ddi.xml
Glycopyrrolate_ddi.xml
Gonadorelin_ddi.xml
Goserelin_ddi.xml
Griseofulvin_ddi.xml
Guanfacine_ddi.xml
Haloperidol_ddi.xml
Halothane_ddi.xml
Heparin_ddi.xml
Heroin_ddi.xml
Hexobarbital_ddi.xml
Hydralazine_ddi.xml
Hydrochlorothiazide_ddi.xml
Hydrocodone_dd

In [5]:
print("# interactions", total_interactions)
print("# total", total_pairs)
print("-------")
print("# effect", drug_dict['effect']['total'])
print("# mechanism", drug_dict['mechanism']['total'])
print("# int", drug_dict['int']['total'])
print("# advise", drug_dict['advise']['total'])
print("# none", drug_dict['none']['total'])

# interactions 3537
# total 25089
-------
# effect 1525
# mechanism 1118
# int 186
# advise 707
# none 21553


In [6]:
for t in ['effect', 'mechanism', 'int', 'advise', 'none']:
    print('-----------------------------------------------')
    print("TYPE", t)
    print("Clue_words")
    total = drug_dict[t]["total"]
    print(sorted(drug_dict[t]['clue_words'].items(), key=lambda item: item[1], reverse=True)[0:20])
    print("Words in sentence")
    print(sorted(drug_dict[t]['sentence_words'].items(), key=lambda item: item[1], reverse=True)[0:20])
    print("E1 under")
    print(sorted(drug_dict[t]['e1_under'].items(), key=lambda item: item[1], reverse=True)[0:20])
    print("E2 under")
    print(sorted(drug_dict[t]['e2_under'].items(), key=lambda item: item[1], reverse=True)[0:20])
    print("e1_e2_under_same_verb", drug_dict[t]['e1_e2_under_same_verb'], drug_dict[t]['e1_e2_under_same_verb']/total, '%')
    print("e1_e2_under_same_word", drug_dict[t]['e1_e2_under_same_word'], drug_dict[t]['e1_e2_under_same_word']/total, '%')
    print("e1_under_e2", drug_dict[t]['e1_under_e2'], drug_dict[t]['e1_under_e2']/total*100, '%')
    print("e2_under_e1", drug_dict[t]['e2_under_e1'], drug_dict[t]['e2_under_e1']/total*100, '%')
    print("total", total)
    print('-----------------------------------------------')

-----------------------------------------------
TYPE effect
Clue_words
[('may', 418), ('drugs', 201), ('response', 171), ('acid', 163), ('effects', 161), ('effect', 157), ('include', 144), ('certain', 108), ('agents', 107), ('antibiotics', 101), ('including', 97), ('alcohol', 96), ('e.g.', 90), ('administered', 87), ('ie', 79), ('man', 79), ('anesthetics', 79), ('enhance', 75), ('steroids', 75), ('patients', 74)]
Words in sentence
[('may', 819), ('drugs', 475), ('effects', 339), ('acid', 292), ('agents', 261), ('patients', 254), ('The', 229), ('effect', 220), ('oral', 199), ('increase', 187), ('sodium', 186), ('inhibitors', 186), ('including', 184), ('action', 179), ('response', 179), ('anesthetics', 176), ('CNS', 164), ('prothrombin', 158), ('e.g.', 157), ('use', 156)]
E1 under
[('', 333), ('use', 91), ('response', 81), ('action', 54), ('effect', 48), ('administration', 48), ('agent', 44), ('enhance', 28), ('decrease', 26), ('increase', 26), ('administer', 25), ('have', 21), ('receive

In [8]:
word_to_check = 'response'
dict_to_check = 'e1_under' # clue_words, sentence_words, e1_under, e2_under

for t in ['effect', 'mechanism', 'int', 'advise', 'none']:
    print('--------------------')
    print(t)
    if word_to_check in drug_dict[t][dict_to_check]:
        print(drug_dict[t][dict_to_check][word_to_check])
    else:
        print(0)
    print(drug_dict[t]['total'])

--------------------
effect
81
1525
--------------------
mechanism
0
1118
--------------------
int
0
186
--------------------
advise
0
707
--------------------
none
14
21553
